<div align="center">
    
# Ski Pricing Strategy     
# 5.0 Modeling

## 5.1 Table of Contents <a id="5.1_Table_of_Contents"></a>
* [5.1 Table of Contents](#5.1_Table_of_Contents)
* [5.2 Introduction](#5.2_Introduction)
* [5.3 Library Imports](#5.3_Library_Imports)
* [5.4 Data Loading](#5.4_Data_Loading)
* [5.5 Model Training](#5.5_Model_Training)
  * [5.5.1 Linear Regression Models](#5.5.1_Linear_Regression_Models)
  * [5.5.2 Tree-Based Models](#5.5.2_Tree_Based_Models)
* [5.6 Model Tuning](#5.6_Model_Tuning)
* [5.7 Final Model](#5.7_Final_Model)
* [5.8 Summary](#5.8_Summary)

## 5.2 Introduction <a id="5.2_Introduction"></a>

This notebook builds predictive models for ski resort weekend ticket prices using the processed and feature-engineered dataset.
We’ll start by training baseline and advanced models, evaluate them using R², RMSE, and MAE, and select the model that offers the best trade-off between performance and generalization.

At the end:

The final model and evaluation metrics will be saved for reuse in the next notebook (sys_06_model_evaluation.ipynb).

The trained model will be applied to estimate Big Mountain Resort’s optimal price.

## 5.3 Library Imports <a id="5.3_Library_Imports"></a>

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from joblib import load, dump

# Models
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# Metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Model fine-tuning
from sklearn.model_selection import GridSearchCV, KFold

# Visualization
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

## 5.4 Data Loading <a id="5.4_Data_Loading"></a>

In [2]:
# load artifacts, training set, validation set and target resort

art = Path("../artifacts")

X_train = np.load(art / "X_train_tf.npy")
X_val   = np.load(art / "X_val_tf.npy")
y_train = pd.read_csv(art / "y_train.csv").squeeze()
y_val   = pd.read_csv(art / "y_val.csv").squeeze()

feature_names = pd.read_csv(art / "feature_names.csv").squeeze().tolist()

print("Data Loaded")
print("Train:", X_train.shape, "| Val:", X_val.shape)
print("y_train mean:", round(y_train.mean(), 2))

Data Loaded
Train: (216, 92) | Val: (55, 92)
y_train mean: 64.32


## 5.5 Model Training <a id="5.5_Model_Training"></a>

This project is a regression problem. The goal is to recommend an appropriate price for the target resort based on its features. Linear models are first applied, including **Linear Regression**, **Ridge Regression**, and **Lasso Regression**. Tree-based models—such as **Decision Trees**, **Random Forests**, and **Gradient Boosting Regressors**—are then used to capture more complex patterns in the data.

In [3]:
# define an evalation metric
def evaluate(model, X_val, y_val):
    preds = model.predict(X_val)
    
    mae = mean_absolute_error(y_val, preds)
    mse = mean_squared_error(y_val, preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_val, preds)
    
    return mae, rmse, r2

### 5.5.1 Linear Regression Models <a id="5.5.1_Linear_Regression_Models"></a>

In [4]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

lr_mae, lr_rmse, lr_r2 = evaluate(lr_model, X_val, y_val)
print("Linear Regression Results:")
print(f"  MAE:  {lr_mae:.3f}")
print(f"  RMSE: {lr_rmse:.3f}")
print(f"  R2:   {lr_r2:.3f}")

Linear Regression Results:
  MAE:  7.842
  RMSE: 10.917
  R2:   0.782


In [5]:
# Ridge Regression
ridge_model = Ridge(alpha=1.0)
ridge_model.fit(X_train, y_train)

ridge_mae, ridge_rmse, ridge_r2 = evaluate(ridge_model, X_val, y_val)
print("Ridge Regression Results:")
print(f"  MAE:  {ridge_mae:.3f}")
print(f"  RMSE: {ridge_rmse:.3f}")
print(f"  R2:   {ridge_r2:.3f}")

Ridge Regression Results:
  MAE:  7.633
  RMSE: 10.612
  R2:   0.794


In [6]:
# Lasso Regression
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X_train, y_train)

lasso_mae, lasso_rmse, lasso_r2 = evaluate(lasso_model, X_val, y_val)
print("Lasso Regression Results:")
print(f"  MAE:  {lasso_mae:.3f}")
print(f"  RMSE: {lasso_rmse:.3f}")
print(f"  R2:   {lasso_r2:.3f}")

Lasso Regression Results:
  MAE:  7.847
  RMSE: 10.913
  R2:   0.782


In [7]:
#ElasticNet Regression
en_model = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=17)
en_model.fit(X_train, y_train)

en_mae, en_rmse, en_r2 = evaluate(en_model, X_val, y_val)

print("ElasticNet Regression Results:")
print(f"  MAE:  {en_mae:.3f}")
print(f"  RMSE: {en_rmse:.3f}")
print(f"  R2:   {en_r2:.3f}")

ElasticNet Regression Results:
  MAE:  7.555
  RMSE: 10.958
  R2:   0.780


### Note:
```text
Model                   MAE       RMSE      R²
-------------------------------------------------
Linear Regression      7.842     10.917    0.782
Ridge Regression*      7.633     10.612    0.794
Lasso Regression       7.847     10.913    0.782
ElasticNet Regression  7.555     10.958    0.780

Ridge Regression is the best linear model, showing the highest R², the lowest RMSE, and a relatively low MAE.

### 5.5.2 Tree-Based Models <a id="5.5.2_Tree_Based_Models"></a>

In [8]:
# Decision Tree Regressor
dt_model = DecisionTreeRegressor(
    max_depth=None,
    random_state=17
)

dt_model.fit(X_train, y_train)
dt_mae, dt_rmse, dt_r2 = evaluate(dt_model, X_val, y_val)

print("Decision Tree Regression Results:")
print(f"  MAE:  {dt_mae:.3f}")
print(f"  RMSE: {dt_rmse:.3f}")
print(f"  R2:   {dt_r2:.3f}")

Decision Tree Regression Results:
  MAE:  13.836
  RMSE: 18.536
  R2:   0.372


In [9]:
# Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=200,
    max_depth=None,
    random_state=17,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
rf_mae, rf_rmse, rf_r2 = evaluate(rf_model, X_val, y_val)

print("Random Forest Regression Results:")
print(f"  MAE:  {rf_mae:.3f}")
print(f"  RMSE: {rf_rmse:.3f}")
print(f"  R2:   {rf_r2:.3f}")

Random Forest Regression Results:
  MAE:  9.428
  RMSE: 13.148
  R2:   0.684


In [10]:
#Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=3,
    random_state=17
)

gb_model.fit(X_train, y_train)
gb_mae, gb_rmse, gb_r2 = evaluate(gb_model, X_val, y_val)

print("Gradient Boosting Regression Results:")
print(f"  MAE:  {gb_mae:.3f}")
print(f"  RMSE: {gb_rmse:.3f}")
print(f"  R2:   {gb_r2:.3f}")

Gradient Boosting Regression Results:
  MAE:  8.204
  RMSE: 11.439
  R2:   0.761


### Note:
```text
Model                     MAE        RMSE       R²
-------------------------------------------------------
Decision Tree            13.836     18.536     0.372
Random Forest             9.428     13.148     0.694
Gradient Boosting         8.204     11.439     0.761

Gradient Boosting is the best tree-based model, achieving the lowest MAE and RMSE and the highest R². However, the best model is still Regid Regression, which outperforms all other models.

## 5.6 Model Tuning <a id="5.6_Model_Tuning"></a>

In [11]:
# Use Cross-Validation to fine tune Regid Regression

# 1. Model
ridge = Ridge()

# 2. Hyperparameter grid
param_grid = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 50, 100]
}

# 3. Cross-validation setup (5-fold)
cv = KFold(n_splits=5, shuffle=True, random_state=17)

# 4. Grid search + CV together
grid = GridSearchCV(
    estimator=ridge,
    param_grid=param_grid,
    cv=cv,
    scoring="neg_mean_squared_error",  # we will convert to RMSE later
    n_jobs=-1
)

grid.fit(X_train, y_train)

print("Best alpha:", grid.best_params_["alpha"])
print("Best CV RMSE:", np.sqrt(-grid.best_score_))

# 5. Evaluate the best model on validation set
best_ridge = grid.best_estimator_
mae, rmse, r2 = evaluate(best_ridge, X_val, y_val)

print("Tuned Ridge Regression on validation set:")
print(f"  MAE:  {mae:.3f}")
print(f"  RMSE: {rmse:.3f}")
print(f"  R2:   {r2:.3f}")


Best alpha: 100
Best CV RMSE: 13.233398467559041
Tuned Ridge Regression on validation set:
  MAE:  8.374
  RMSE: 11.601
  R2:   0.754


### Note:
```text
Model                       MAE       RMSE      R²
---------------------------------------------------------
Ridge Regression (alpha=1)  7.633     10.612    0.794
Tuned Ridge (alpha=100)     8.374     11.601    0.754

The original Ridge Regression model with alpha = 1 provides the strongest performance among the Ridge configurations. It achieves the lowest RMSE, the highest R², and a lower MAE compared to the tuned model. The tuned Ridge Regression with alpha = 100, selected through cross-validation, shows weaker performance on the validation set, indicating that heavier regularization does not improve prediction accuracy for this dataset. Therefore, Ridge Regression with alpha = 1 is selected as the final linear model, as it offers the best overall balance of error and model fit.

## 5.7 Final Model <a id="5.7_Final_Model"></a>

In [12]:
# Combine train + val into final training set
X_final = np.concatenate([X_train, X_val])
y_final = pd.concat([y_train, y_val])

# Train final Ridge model
final_model = Ridge(alpha=1)
final_model.fit(X_final, y_final)

print("Final model trained on ALL available labeled data.")

Final model trained on ALL available labeled data.


In [13]:
# load features of target resort
X_tgt = np.load(art / "X_tgt_tf.npy")
print("Target features shape:", X_tgt.shape)

Target features shape: (1, 92)


In [14]:
# Predict ticket price for target resort
y_tgt_pred = final_model.predict(X_tgt)

print("Predicted Ticket Price:", round(float(y_tgt_pred[0]), 2))

Predicted Ticket Price: 94.58


## 5.8 Summary<a id='5.8_Summary'></a>

The final Ridge Regression model (α = 1), trained on all available data, predicts a recommended ticket price of 94.58 dollar for the target resort.
The resort’s current ticket price is 81 dollar, which is approximately $13.58 lower than the model’s estimated optimal value.

This indicates that the resort may be underpricing relative to the historical relationships between features such as weather conditions, demand indicators, seasonality, and other engineered attributes.
Adjusting the price closer to the model’s recommendation could potentially improve revenue while remaining consistent with past pricing patterns and market behavior.

In [15]:
#SaveArtifacts
dump(final_model, art / "final_model.joblib")
print("✅ Final model saved to:", art / "final_model.joblib")

✅ Final model saved to: ../artifacts/final_model.joblib
